In [ ]:
import sys
import argparse
##############################################################################
args = {}
args['dataset'] = 'mnist34' # 数据集名称
#args['dataroot'] = '/path/to/dataset'  # 数据集路径
args['workers'] = 2  # 数据加载线程数
args['batchSize'] = 64  # 批处理大小
args['imageSize'] = 28  # 图像尺寸
args['nz'] = 110  # 噪声向量维度 # 100+10
args['ngf'] = 64  # 生成器特征图通道数
args['ndf'] = 64  # 判别器特征图通道数
args['niter'] = 50  # 训练迭代次数
args['lr'] = 0.0002  # 学习率
args['beta1'] = 0.5  # Adam 优化器的 beta1 参数
args['cuda'] = False  # 是否使用 CUDA
args['ngpu'] = 1  # 使用的 GPU 数量
args['netG'] = ''  # 生成器模型文件
args['netD'] = ''  # 判别器模型文件
args['outf'] = '.'  # 输出文件夹
args['manualSeed'] = None  # 随机数种子
args['num_classes'] = 2  # 类别数
args['gpu_id'] = 0  # GPU ID
args['fraction'] = 0.005 # 有监督学习的样本比例
print(args)
##############################################################################

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys
import os
sys.path.append(os.path.abspath("/Users/max/MasterThesis/Training/"))
import torch
from torch.utils.data import DataLoader, Subset, ConcatDataset, TensorDataset
from torchvision import datasets, transforms
import numpy as np
import torch
from torch.utils.data import DataLoader, Subset, ConcatDataset, TensorDataset
from torchvision import datasets, transforms
import numpy as np
print("This is the MNIST dataset with labels 3 and 4.")
print("Imbalanced Ratio: ", args['fraction'])
# 数据预处理
mnist34_transforms = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.1307,), (0.3081,))
])

# 加载 MNIST 数据集
full_train_datasets = datasets.MNIST(root="/Users/max/MasterThesisData/MNIST", train=True, transform=mnist34_transforms, download=True)
full_test_datasets = datasets.MNIST(root="/Users/max/MasterThesisData/MNIST", train=False, transform=mnist34_transforms, download=True)

# 选取标签为 3 和 4 的索引
indices3_train = [i for i in range(len(full_train_datasets)) if full_train_datasets.targets[i] == 3]
indices4_train = [i for i in range(len(full_train_datasets)) if full_train_datasets.targets[i] == 4]

indices3_test = [i for i in range(len(full_test_datasets)) if full_test_datasets.targets[i] == 3]
indices4_test = [i for i in range(len(full_test_datasets)) if full_test_datasets.targets[i] == 4]

# 获取训练集中标签为 3 和 4 的数据
mnist3_train_data = full_train_datasets.data[indices3_train]
mnist3_train_labels = torch.ones(len(indices3_train), dtype=torch.long)  # 标签 3 映射为 1 

mnist4_train_data = full_train_datasets.data[indices4_train]
mnist4_train_labels = torch.zeros(len(indices4_train), dtype=torch.long)  # 标签 4 映射为 0

# 获取测试集中标签为 3 和 4 的数据
mnist3_test_data = full_test_datasets.data[indices3_test]
mnist3_test_labels = torch.ones(len(indices3_test), dtype=torch.long)  # 标签 3 映射为 1 

mnist4_test_data = full_test_datasets.data[indices4_test]
mnist4_test_labels = torch.zeros(len(indices4_test), dtype=torch.long)  # 标签 4 映射为 0

# we can set the imbalanced ratio 0.005, 0.01, 0.02, 0.05, 0.1, 0.2
fraction = int(args['fraction'] * len(mnist3_train_data))  ### control the fraction of the data to be used
selected_indices_4 = np.random.choice(len(mnist4_train_data), fraction, replace=False)

fraction_mnist4_train_data = mnist4_train_data[selected_indices_4]
fraction_mnist4_train_labels = mnist4_train_labels[selected_indices_4]

# 创建最终的训练和测试数据集
Final_train_data = torch.cat([mnist3_train_data, fraction_mnist4_train_data], dim=0)
Final_train_labels = torch.cat([mnist3_train_labels, fraction_mnist4_train_labels], dim=0)

Final_test_data = torch.cat([mnist3_test_data, mnist4_test_data], dim=0)
Final_test_labels = torch.cat([mnist3_test_labels, mnist4_test_labels], dim=0)

# 创建 TensorDataset
Final_train_datasets = TensorDataset(Final_train_data.unsqueeze(1).float() / 255, Final_train_labels)
Final_test_datasets = TensorDataset(Final_test_data.unsqueeze(1).float() / 255, Final_test_labels)

# 数据加载器
train_loader = DataLoader(Final_train_datasets, batch_size=64, shuffle=True, drop_last=True)
test_loader = DataLoader(Final_test_datasets, batch_size=64, shuffle=False, drop_last=True)

# 打印信息
print("Number of label 3 in the final training set: ", len(mnist3_train_data))
print("Number of label 4 in the final training set (after downsampling): ", len(fraction_mnist4_train_data))
print("Number of label 3 in the final test set: ", len(mnist3_test_data))
print("Number of label 4 in the final test set: ", len(mnist4_test_data))

print("Total samples in final training set: ", len(Final_train_datasets))
print("Total samples in final test set: ", len(Final_test_datasets))

print("Number of batches in training set: ", len(train_loader))
print("Number of batches in test set: ", len(test_loader))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# Attention Block
class AttentionBlock(nn.Module):
    def __init__(self, filters):
        """
        :param filters: 输入特征图的通道数(同时也是卷积输出的通道数)
        """
        super(AttentionBlock, self).__init__()
        # 与 Keras 中的 Conv2D(filters, kernel_size=1, padding='same') 对应
        # PyTorch 中 padding=0 就相当于 'same'（仅当 kernel_size=1 时）
        self.query_conv = nn.Conv2d(filters, filters, kernel_size=1, padding=0)
        self.key_conv   = nn.Conv2d(filters, filters, kernel_size=1, padding=0)
        self.value_conv = nn.Conv2d(filters, filters, kernel_size=1, padding=0)

    def forward(self, x):
        """
        x 的形状一般是 (batch_size, filters, H, W)
        """
        # 1. 分别得到 query, key, value
        query = F.relu(self.query_conv(x))
        key   = F.relu(self.key_conv(x))
        value = F.relu(self.value_conv(x))

        # 2. 计算注意力图: 先元素乘，再对通道维度 (dim=1) 求和
        attention_map = query * key                  # 形状 (N, filters, H, W)
        attention_map = torch.sum(attention_map, dim=1, keepdim=True)  
        # 现在 attention_map 的形状是 (N, 1, H, W)

        # 3. 对空间维度 (H, W) 做 softmax
        # 先展平再 softmax，再 reshape 回去
        N, _, H, W = attention_map.shape
        attention_map = attention_map.view(N, 1, -1)         # (N, 1, H*W)
        attention_map = F.softmax(attention_map, dim=-1)     # 在 H*W 上做 softmax
        attention_map = attention_map.view(N, 1, H, W)       # (N, 1, H, W)

        # 4. 注意力加权 value，并与原输入相加
        attended_value = attention_map * value
        output = x + attended_value

        return output

# 定义用于MNIST生成器的网络
class _netG_MNIST(nn.Module):
    def __init__(self, ngpu, nz):
        super(_netG_MNIST, self).__init__()  # 调用父类的构造函数
        self.ngpu = ngpu  # 保存GPU数量
        self.nz = nz      # 噪声向量的维度（例如：噪声+标签拼接后为110）

        # 全连接层：将输入噪声向量映射为256*7*7个特征
        self.fc = nn.Linear(nz, 256 * 7 * 7)  # 输入维度为nz，输出维度为256*7*7

        # 转置卷积层1：将256通道、7x7的特征图转换为128通道、14x14的特征图
        self.tconv1 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),  # 转置卷积，尺寸放大2倍
            nn.BatchNorm2d(128),         # 对128个通道做批归一化
            nn.ReLU(True)                # 使用ReLU激活函数
        )

        # 转置卷积层2：将128通道、14x14的特征图转换为1通道、28x28的图像
        self.tconv2 = nn.Sequential(
            nn.ConvTranspose2d(128, 1, kernel_size=4, stride=2, padding=1, bias=False),  # 转置卷积，尺寸放大2倍
            nn.Tanh()                  # 使用Tanh激活函数，将输出映射到[-1,1]之间
        )

    def forward(self, input):
        # 若使用CUDA且多GPU，则采用数据并行方式
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            input = input.view(-1, self.nz)                           # 将输入展平为二维张量[batch_size, nz]
            fc = nn.parallel.data_parallel(self.fc, input, range(self.ngpu))  # 并行通过全连接层
            fc = fc.view(-1, 256, 7, 7)                               # 重塑为[batch_size, 256, 7, 7]的特征图
            tconv1 = nn.parallel.data_parallel(self.tconv1, fc, range(self.ngpu))  # 并行通过第一个转置卷积层
            tconv2 = nn.parallel.data_parallel(self.tconv2, tconv1, range(self.ngpu))  # 并行通过第二个转置卷积层
            output = tconv2                                         # 得到生成的图像
        else:
            input = input.view(-1, self.nz)                           # 将输入展平为二维张量[batch_size, nz]
            fc = self.fc(input)                                       # 通过全连接层
            fc = fc.view(-1, 256, 7, 7)                               # 重塑为[batch_size, 256, 7, 7]的特征图
            tconv1 = self.tconv1(fc)                                  # 通过第一个转置卷积层
            tconv2 = self.tconv2(tconv1)                              # 通过第二个转置卷积层
            output = tconv2                                         # 得到生成的图像
        return output  # 返回生成的28x28单通道图像

# # 定义用于MNIST判别器的网络
class _netD_MNIST(nn.Module):
    def __init__(self, ngpu, num_classes=2):
        super(_netD_MNIST, self).__init__()  # 调用父类的构造函数
        self.ngpu = ngpu  # 保存GPU数量

        # 卷积层1：将输入1通道图像转换为16通道，输出尺寸为14x14（28x28经步长为2降采样）
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1, bias=False),  # 卷积操作
            nn.LeakyReLU(0.2, inplace=True),  # LeakyReLU激活函数，负半轴斜率为0.2
            nn.Dropout(0.5, inplace=False)      # Dropout防止过拟合
        )

        # 卷积层2：保持空间尺寸不变，增加通道数到32
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1, bias=False),  # 卷积操作
            nn.BatchNorm2d(32),             # 对32个通道做批归一化
            nn.LeakyReLU(0.2, inplace=True),  # LeakyReLU激活函数
            nn.Dropout(0.5, inplace=False)      # Dropout防止过拟合
        )

        # 卷积层3：将特征图尺寸减半（14x14→7x7），通道数增至64
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1, bias=False),  # 卷积操作
            nn.BatchNorm2d(64),             # 对64个通道做批归一化
            nn.LeakyReLU(0.2, inplace=True),  # LeakyReLU激活函数
            nn.Dropout(0.5, inplace=False)      # Dropout防止过拟合
        )

        # 卷积层4：保持7x7尺寸，通道数增至128
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False),  # 卷积操作
            nn.BatchNorm2d(128),            # 对128个通道做批归一化
            nn.LeakyReLU(0.2, inplace=True),  # LeakyReLU激活函数
            nn.Dropout(0.5, inplace=False)      # Dropout防止过拟合
        )

        # 卷积层5：将特征图尺寸减半（7x7→4x4，因7+2-3=6，6//2+1=4），通道数增至256
        self.conv5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1, bias=False),  # 卷积操作
            nn.BatchNorm2d(256),            # 对256个通道做批归一化
            nn.LeakyReLU(0.2, inplace=True),  # LeakyReLU激活函数
            nn.Dropout(0.5, inplace=False)      # Dropout防止过拟合
        )

        # 卷积层6：保持特征图尺寸为4x4，通道数增至512
        self.conv6 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False),  # 卷积操作
            nn.BatchNorm2d(512),            # 对512个通道做批归一化
            nn.LeakyReLU(0.2, inplace=True),  # LeakyReLU激活函数
            nn.Dropout(0.5, inplace=False)      # Dropout防止过拟合
        )

        # 全连接层：用于判别图像的真伪（discriminator）
        self.fc_dis = nn.Linear(4 * 4 * 512, 1)  # 将4x4x512的特征展平后映射到1个输出
        # 全连接层：用于辅助分类器，预测图像类别
        self.fc_aux = nn.Linear(4 * 4 * 512, num_classes)  # 将特征映射到类别数
        # Softmax层：用于计算辅助分类器输出的概率分布
        self.softmax = nn.Softmax(dim=1)  # 指定在类别维度上做归一化
        # Sigmoid层：用于计算判别器输出的真伪概率
        self.sigmoid = nn.Sigmoid()  # 将输出映射到[0,1]之间

    def forward(self, input):
        # 若使用CUDA且多GPU，则采用数据并行方式
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            conv1 = nn.parallel.data_parallel(self.conv1, input, range(self.ngpu))  # 第一卷积层并行计算
            conv2 = nn.parallel.data_parallel(self.conv2, conv1, range(self.ngpu))  # 第二卷积层并行计算
            conv3 = nn.parallel.data_parallel(self.conv3, conv2, range(self.ngpu))  # 第三卷积层并行计算
            conv4 = nn.parallel.data_parallel(self.conv4, conv3, range(self.ngpu))  # 第四卷积层并行计算
            conv5 = nn.parallel.data_parallel(self.conv5, conv4, range(self.ngpu))  # 第五卷积层并行计算
            conv6 = nn.parallel.data_parallel(self.conv6, conv5, range(self.ngpu))  # 第六卷积层并行计算
            flat6 = conv6.view(-1, 4 * 4 * 512)  # 将卷积输出展平成二维张量
            fc_dis = nn.parallel.data_parallel(self.fc_dis, flat6, range(self.ngpu))  # 判别器全连接层并行计算
            fc_aux = nn.parallel.data_parallel(self.fc_aux, flat6, range(self.ngpu))  # 辅助分类器全连接层并行计算
        else:
            conv1 = self.conv1(input)       # 通过第一卷积层
            conv2 = self.conv2(conv1)       # 通过第二卷积层
            conv3 = self.conv3(conv2)       # 通过第三卷积层
            conv4 = self.conv4(conv3)       # 通过第四卷积层
            conv5 = self.conv5(conv4)       # 通过第五卷积层
            conv6 = self.conv6(conv5)       # 通过第六卷积层
            flat6 = conv6.view(-1, 4 * 4 * 512)  # 将卷积输出展平成二维张量
            fc_dis = self.fc_dis(flat6)     # 通过判别器全连接层
            fc_aux = self.fc_aux(flat6)     # 通过辅助分类器全连接层

        classes = self.softmax(fc_aux)  # 计算辅助分类器的类别概率分布
        realfake = self.sigmoid(fc_dis).view(-1, 1).squeeze(1)  # 计算判别器的真伪概率，并调整输出形状
        return realfake, classes  # 返回真伪判断结果和类别预测


In [ ]:
from __future__ import print_function  # 引入未来版本的打印函数，确保兼容 Python 3 语法
import argparse  # 导入 argparse 模块，用于解析命令行参数
import os  # 导入 os 模块，用于文件和目录操作
import numpy as np  # 导入 numpy 模块并简写为 np，用于数值计算
import random  # 导入 random 模块，用于生成随机数
import torch  # 导入 PyTorch 库
import torch.nn as nn  # 导入 PyTorch 神经网络模块，并简写为 nn
import torch.nn.parallel  # 导入 PyTorch 的并行计算模块
import torch.backends.cudnn as cudnn  # 导入 cuDNN 后端，用于加速卷积计算
import torch.optim as optim  # 导入 PyTorch 优化器模块，并简写为 optim
import torch.utils.data  # 导入 PyTorch 数据加载工具模块
import torchvision.datasets as dset  # 导入 torchvision 数据集模块，并简写为 dset
import torchvision.transforms as transforms  # 导入 torchvision 数据预处理模块，并简写为 transforms
import torchvision.utils as vutils  # 导入 torchvision 工具模块，用于处理图像数据，并简写为 vutils
#from torch.autograd import Variable  # 从 torch.autograd 模块中导入 Variable 类（用于封装张量，旧版本 PyTorch 使用）
from utils import weights_init, compute_acc  # 从 utils 模块导入自定义权重初始化和计算准确率的函数
from network import _netG, _netD, _netD_CIFAR10, _netG_CIFAR10, _netD_MNIST, _netG_MNIST  # 从 network 模块导入生成器和判别器网络
#from folder import ImageFolder  # 从 folder 模块导入 ImageFolder 类，用于加载自定义图像数据
from Get_datasets import get_datasets # 从 Get_datasets 模块导入 get_datasets 函数，用于加载 MNIST-3/4 数据集

In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []
# 如果只使用一个 GPU，则指定使用的 GPU ID
if args['ngpu'] == 1:
    os.environ['CUDA_VISIBLE_DEVICES'] = str(args['gpu_id'])  # 设置环境变量，只显示指定的 GPU

try:
    os.makedirs(args['outf'])  # 尝试创建输出文件夹
except OSError:
    pass  # 如果输出文件夹已存在，则忽略异常

if args['manualSeed'] is None:
    args['manualSeed'] = random.randint(1, 10000)  # 如果未手动设置随机种子，则随机生成一个
print("Random Seed: ", args['manualSeed'])  # 打印随机种子
random.seed(args['manualSeed'])  # 设置 Python 内置随机数种子
torch.manual_seed(args['manualSeed'])  # 设置 PyTorch 随机数种子
if args['cuda']:
    torch.cuda.manual_seed_all(args['manualSeed'])  # 如果使用 CUDA，则设置所有 GPU 的随机数种子

cudnn.benchmark = True  # 启用 cuDNN benchmark 模式以选择最优算法，加快训练

if torch.cuda.is_available() and not args['cuda']:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")  # 如果有 CUDA 设备但未启用 CUDA，则发出警告

In [ ]:
# 设置一些超参数
ngpu = int(args['ngpu'])  # 获取 GPU 数量
nz = int(args['nz'])  # 获取潜在向量 z 的维度
ngf = int(args['ngf'])  # 获取生成器特征图基数
ndf = int(args['ndf'])  # 获取判别器特征图基数
num_classes = int(args['num_classes'])  # 获取 AC-GAN 的类别数
nc = 1  # 定义输入图像的通道数（3 表示 RGB 图像）


In [ ]:
# 定义生成器网络并初始化权重
if args['dataset'] == 'imagenet':
    netG = _netG(ngpu, nz)  # 如果数据集为 imagenet，使用 _netG 生成器
elif args['dataset'] == 'cifar10':
    netG = _netG_CIFAR10(ngpu, nz)  # 如果数据集为 cifar10，使用 _netG_CIFAR10 生成器
else:
    netG = _netG_MNIST(ngpu, nz)
netG.apply(weights_init)  # 对生成器网络中的所有模块应用自定义权重初始化
if args['netG'] != '':  # 如果提供了预训练生成器模型路径，则加载模型参数
    netG.load_state_dict(torch.load(args['netG']))
print(netG)  # 打印生成器网络结构

In [ ]:
# 定义判别器网络并初始化权重
if args['dataset'] == 'imagenet':
    netD = _netD(ngpu, num_classes)  # 如果数据集为 imagenet，使用 _netD 判别器
elif args['dataset'] == 'cifar10':
    netD = _netD_CIFAR10(ngpu, num_classes)  # 如果数据集为 cifar10，使用 _netD_CIFAR10 判别器
else:
    netD = _netD_MNIST(ngpu, num_classes) # 如果数据集为 mnist，使用 _netD_MNIST 判别器
netD.apply(weights_init)  # 对判别器网络中的所有模块应用自定义权重初始化
if args['netD'] != '':  # 如果提供了预训练判别器模型路径，则加载模型参数
    netD.load_state_dict(torch.load(args['netD']))
print(netD)  # 打印判别器网络结构

In [ ]:
# 定义损失函数
dis_criterion = nn.BCELoss()  # 使用二元交叉熵损失函数计算判别器的真伪损失
aux_criterion = nn.NLLLoss()  # 使用负对数似然损失函数计算辅助分类器的损失


In [ ]:
# 定义张量占位符
input = torch.FloatTensor(args['batchSize'], 1, args['imageSize'], args['imageSize'])  # 创建存储输入图像的张量，占位尺寸为 [batchSize, 1, imageSize, imageSize]
noise = torch.FloatTensor(args['batchSize'], nz, 1, 1)  # 创建存储生成器噪声的张量，占位尺寸为 [batchSize, nz, 1, 1]
eval_noise = torch.FloatTensor(args['batchSize'], nz, 1, 1).normal_(0, 1)  # 创建评估用噪声张量，并用标准正态分布初始化
dis_label = torch.FloatTensor(args['batchSize'])  # 创建存储判别器标签的张量，占位尺寸为 [batchSize]
aux_label = torch.LongTensor(args['batchSize'])  # 创建存储辅助分类标签的张量，占位尺寸为 [batchSize]
real_label = 1  # 定义真实图像标签值为 1
fake_label = 0

In [ ]:
# 如果使用 CUDA，则将网络和张量移动到 GPU 上
if args['cuda']:
    netD.cuda()  # 将判别器移动到 GPU
    netG.cuda()  # 将生成器移动到 GPU
    dis_criterion.cuda()  # 将判别器损失函数移动到 GPU
    aux_criterion.cuda()  # 将辅助分类损失函数移动到 GPU
    input, dis_label, aux_label = input.cuda(), dis_label.cuda(), aux_label.cuda()  # 将输入和标签张量移动到 GPU
    noise, eval_noise = noise.cuda(), eval_noise.cuda()  # 将噪声张量移动到 GPU


In [ ]:
# 直接使用 tensor，不再需要封装为 Variable
# 假设 input, noise, eval_noise, dis_label, aux_label 本身已经是 Tensor
# 如果需要使某个张量支持梯度计算，可以设置 requires_grad=True
input = input       # 输入图像张量
noise = noise       # 生成器噪声张量
eval_noise = eval_noise  # 评估用噪声张量
dis_label = dis_label    # 判别器标签张量
aux_label = aux_label    # 辅助分类标签张量

In [ ]:
# 生成评估用的噪声和标签
eval_noise_ = np.random.normal(0, 1, (args['batchSize'], nz))  # 使用正态分布生成评估噪声，尺寸为 [batchSize, nz]
eval_label = np.random.randint(0, num_classes, args['batchSize'])  # 随机生成评估标签，取值范围在 0 到 num_classes-1
eval_onehot = np.zeros((args['batchSize'], num_classes))  # 创建评估用的 one-hot 编码矩阵，尺寸为 [batchSize, num_classes]
eval_onehot[np.arange(args['batchSize']), eval_label] = 1  # 根据生成的标签设置 one-hot 编码
eval_noise_[np.arange(args['batchSize']), :num_classes] = eval_onehot[np.arange(args['batchSize'])]  # 将 one-hot 编码嵌入评估噪声的前 num_classes 维
eval_noise_ = (torch.from_numpy(eval_noise_))  # 将 numpy 数组转换为 PyTorch 张量
eval_noise.data.copy_(eval_noise_.view(args['batchSize'], nz, 1, 1))  # 将生成的评估噪声复制到 eval_noise 变量中，并调整形状


In [ ]:
# 设置优化器
optimizerD = optim.Adam(netD.parameters(), lr=args['lr'], betas=(args['beta1'], 0.999))  # 使用 Adam 优化器优化判别器参数
optimizerG = optim.Adam(netG.parameters(), lr=args['lr'], betas=(args['beta1'], 0.999))  # 使用 Adam 优化器优化生成器参数

avg_loss_D = 0.0  # 初始化判别器平均损失为 0
avg_loss_G = 0.0  # 初始化生成器平均损失为 0
avg_loss_A = 0.0  # 初始化辅助分类器平均准确率为 0

In [ ]:
for epoch in range(args['niter']):  # 循环遍历每个 epoch（训练轮数）
    for i, data in enumerate(train_loader, 0):  # 遍历数据加载器中的每个批次
        ############################
        # (1) 更新判别器网络：最大化 log(D(x)) + log(1 - D(G(z)))
        ############################
        # 用真实数据训练判别器
        netD.zero_grad()  # 清空判别器的梯度
        real_cpu, label = data  # 获取当前批次的真实图像和对应标签
        batch_size = real_cpu.size(0)  # 获取当前批次的样本数
        if args['cuda']:
            real_cpu = real_cpu.cuda()  # 如果使用 CUDA，将真实图像移动到 GPU
        input.data.resize_as_(real_cpu).copy_(real_cpu)  # 调整 input 张量大小以匹配真实图像，并复制图像数据
        dis_label.data.resize_(batch_size).fill_(real_label)  # 将判别器标签调整为当前批次大小，并填充为真实标签
        aux_label.data.resize_(batch_size).copy_(label)  # 将辅助分类标签调整为当前批次大小，并复制真实标签
        dis_output, aux_output = netD(input)  # 将真实图像输入判别器，获得真伪输出和辅助分类输出

        dis_errD_real = dis_criterion(dis_output, dis_label)  # 计算真实图像的判别器损失
        aux_errD_real = aux_criterion(aux_output, aux_label)  # 计算真实图像的辅助分类损失
        errD_real = dis_errD_real + aux_errD_real  # 计算真实图像总损失（判别器损失 + 辅助分类损失）
        errD_real.backward()  # 对真实图像损失进行反向传播，计算梯度
        D_x = dis_output.data.mean()  # 计算真实图像输出的平均值，用于监控

        # 计算当前批次的分类准确率
        accuracy = compute_acc(aux_output, aux_label)  # 调用 compute_acc 函数计算辅助分类器的准确率

        # 用生成的假数据训练判别器
        noise.data.resize_(batch_size, nz, 1, 1).normal_(0, 1)  # 生成随机噪声，并调整其形状为 [batchSize, nz, 1, 1]
        label = np.random.randint(0, num_classes, batch_size)  # 随机生成当前批次的标签，用作条件输入
        noise_ = np.random.normal(0, 1, (batch_size, nz))  # 生成噪声数组，尺寸为 [batchSize, nz]
        class_onehot = np.zeros((batch_size, num_classes))  # 创建一个全零矩阵，用于生成 one-hot 编码
        class_onehot[np.arange(batch_size), label] = 1  # 根据随机标签生成 one-hot 编码
        noise_[np.arange(batch_size), :num_classes] = class_onehot[np.arange(batch_size)]  # 将 one-hot 编码嵌入噪声的前 num_classes 维
        # noise_ = (torch.from_numpy(noise_))  # 将 numpy 数组转换为 PyTorch 张量
        # noise.data.copy_(noise_.view(batch_size, nz, 1, 1))  # 将处理好的噪声复制到 noise 变量中，并调整形状
        noise_ = torch.from_numpy(noise_)
        noise.data.copy_(noise_.view(batch_size, nz, 1, 1)) # 将处理好的噪声复制到 noise 变量中，并调整形状
        aux_label = torch.from_numpy(label).to(device)  # device could be 'cuda' if needed  # 将生成的标签转换为张量并复制到辅助标签变量中

        fake = netG(noise)  # 使用生成器生成假图像
        dis_label.data.fill_(fake_label)  # 将判别器标签全部设置为虚假标签
        dis_output, aux_output = netD(fake.detach())  # 将生成的假图像输入判别器，detach() 防止梯度传回生成器
        dis_errD_fake = dis_criterion(dis_output, dis_label)  # 计算假图像的判别器损失
        aux_errD_fake = aux_criterion(aux_output, aux_label)  # 计算假图像的辅助分类损失
        errD_fake = dis_errD_fake + aux_errD_fake  # 计算假图像总损失（判别器损失 + 辅助分类损失）
        errD_fake.backward()  # 对假图像损失进行反向传播，计算梯度
        D_G_z1 = dis_output.data.mean()  # 计算生成假图像的判别器输出平均值，用于监控
        errD = errD_real + errD_fake  # 计算判别器总损失（真实 + 假数据的损失之和）
        optimizerD.step()  # 更新判别器参数

        ############################
        # (2) 更新生成器网络：最大化 log(D(G(z)))
        ############################
        netG.zero_grad()  # 清空生成器的梯度
        dis_label.data.fill_(real_label)  # 将判别器标签设置为真实标签（生成器目标是欺骗判别器）
        dis_output, aux_output = netD(fake)  # 将生成的假图像输入判别器，获得输出
        dis_errG = dis_criterion(dis_output, dis_label)  # 计算生成器的判别器损失
        aux_errG = aux_criterion(aux_output, aux_label)  # 计算生成器的辅助分类损失
        errG = dis_errG + aux_errG  # 计算生成器总损失（判别器损失 + 辅助分类损失）
        errG.backward()  # 对生成器损失进行反向传播，计算梯度
        D_G_z2 = dis_output.data.mean()  # 计算生成图像在判别器中的输出平均值，用于监控
        optimizerG.step()  # 更新生成器参数

        # 计算当前的平均损失和平均准确率
        curr_iter = epoch * len(train_loader) + i  # 计算当前迭代的总步数
        all_loss_G = avg_loss_G * curr_iter  # 累加之前的生成器总损失
        all_loss_D = avg_loss_D * curr_iter  # 累加之前的判别器总损失
        all_loss_A = avg_loss_A * curr_iter  # 累加之前的辅助分类准确率总和
        all_loss_G += errG.item()  # 加入当前生成器损失
        all_loss_D += errD.item() # 加入当前判别器损失
        all_loss_A += accuracy  # 加入当前准确率
        avg_loss_G = all_loss_G / (curr_iter + 1)  # 计算新的生成器平均损失
        avg_loss_D = all_loss_D / (curr_iter + 1)  # 计算新的判别器平均损失
        avg_loss_A = all_loss_A / (curr_iter + 1)  # 计算新的辅助分类平均准确率

        # 打印当前训练状态，包括损失、判别器输出和准确率
        print('[%d/%d][%d/%d] Loss_D: %.4f (%.4f) Loss_G: %.4f (%.4f) D(x): %.4f D(G(z)): %.4f / %.4f Acc: %.4f (%.4f)'
              % (epoch, args['niter'], i, len(train_loader),
                 errD.item(), avg_loss_D, errG.item(), avg_loss_G, D_x, D_G_z1, D_G_z2, accuracy, avg_loss_A))
        if i % 100 == 0:  # 每100个批次执行一次保存操作
            vutils.save_image(
                real_cpu, '%s/real_samples.png' % args['outf'])  # 保存当前批次的真实图像样本到指定文件夹
            print('Label for eval = {}'.format(eval_label))  # 打印评估时使用的标签
            fake = netG(eval_noise)  # 用评估噪声生成假图像
            vutils.save_image(
                fake.data,
                '%s/fake_samples_epoch_%03d.png' % (args['outf'], epoch)  # 保存生成器生成的假图像样本，文件名中包含当前 epoch
            )

    # 每个 epoch 结束后，保存当前的模型检查点
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (args['outf'], epoch))  # 保存生成器参数
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (args['outf'], epoch))  # 保存判别器参数